In [11]:
import os
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

In [12]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# oss

def round_to_nearest_integer(series):
  return series.round().astype(int)

data_dir = './dms_data/'
train_data_path = data_dir + '/train'
test_data_path = data_dir + '/test'

train_files = os.listdir(train_data_path)
train_df_list = []
for file in train_files:
  if file.endswith('.csv'):
    df = pd.read_csv(os.path.join(train_data_path, file))
    train_df_list.append(df)
train_df = pd.concat(train_df_list, ignore_index=True)

test_files = os.listdir(test_data_path)
test_df_list = []
for file in test_files:
  if file.endswith('.csv'):
    df = pd.read_csv(os.path.join(test_data_path, file))
    test_df_list.append(df)
test_df = pd.concat(test_df_list, ignore_index=True)


X_train = train_df.drop(columns=["timestamp", "oss", "Sleepiness"], errors="ignore")
y_train = round_to_nearest_integer(train_df['oss'])

X_test = test_df.drop(columns=["timestamp", "oss", "Sleepiness"], errors="ignore")
y_test = round_to_nearest_integer(test_df['oss'])


X_train = X_train.dropna()
y_train = y_train[X_train.index]

X_test = X_test.dropna()
y_test = y_test[X_test.index]

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Precision, Recall, F1-Scoreを計算
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Confusion Matrixを計算
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:\n", conf_matrix)



Accuracy: 0.6074074074074074
Accuracy: 0.6074074074074074
Precision: 0.411715586522756
Recall: 0.38492287614263593
F1-Score: 0.3840879307281907
Confusion Matrix:
 [[  0   2   3   0   0]
 [  0  88  67   6   0]
 [  0  24 196  33   4]
 [  0   0  31  40   7]
 [  0   0   8  27   4]]


/home/buntin/jupyterlab/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# One-Hot Encoding for Neural Network
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))

# Neural Network Model
model = Sequential()
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(y_train_encoded.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train_encoded, epochs=10, batch_size=32)

# Neural Network Prediction
y_pred_nn_prob = model.predict(X_test)
y_pred_nn = np.argmax(y_pred_nn_prob, axis=1)

# Neural Network Accuracy
accuracy_nn = accuracy_score(y_test, y_pred_nn)
print("Neural Network Accuracy:", accuracy_nn)


/home/buntin/jupyterlab/lib64/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: Found unknown categories [1] in column 0 during transform

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


all_y = pd.concat([y_train, y_test])
all_y_encoded = label_encoder.fit_transform(all_y)

y_train_encoded = all_y_encoded[:len(y_train)]
y_test_encoded = all_y_encoded[len(y_train):]

# LightGBMでの学習と予測
dtrain = lgb.Dataset(X_train, label=y_train_encoded)
params = {
    'objective': 'multiclass',
    'num_class': len(np.unique(y_train_encoded)),
    'boosting_type': 'gbdt',
    'metric': 'multi_logloss',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'verbose': -1
}
clf_lgb = lgb.train(params, dtrain, 100)
y_pred_lgb_encoded = clf_lgb.predict(X_test)
y_pred_lgb_encoded = np.argmax(y_pred_lgb_encoded, axis=1)

y_pred_lgb = label_encoder.inverse_transform(y_pred_lgb_encoded)

# LightGBMの精度計算
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
print("LightGBM Accuracy:", accuracy_lgb)


[LightGBM] [Fatal] Label must be in [0, 4), but found 4 in label


LightGBMError: Label must be in [0, 4), but found 4 in label